<a href="https://colab.research.google.com/github/esdrasantos/statistical-concepts-and-practice/blob/main/practice_entropy_information_gain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [294]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [295]:
import os  # Operational System commands, usually in-out functions
import re  # Regex package

In [296]:
dir = '/content/drive/MyDrive/faturas_nu_csv'

arquivos = [arquivo for arquivo in os.listdir(dir)]

padrao = f"_(.*?)-"

arquivosord = sorted(arquivos, key=lambda x: int(re.findall(padrao, x)[0]))
arquivosord = sorted(arquivosord, key=lambda x: int(x.split('-')[1]))
arquivosord

['Nubank_2025-01-11.csv',
 'Nubank_2025-02-11.csv',
 'Nubank_2025-03-11.csv',
 'Nubank_2025-04-11.csv',
 'Nubank_2025-05-11.csv',
 'Nubank_2025-06-11.csv',
 'Nubank_2025-07-11.csv',
 'Nubank_2025-08-11.csv',
 'Nubank_2025-09-11.csv',
 'Nubank_2025-10-11.csv',
 'Nubank_2025-11-11.csv']

In [297]:
import pandas as pd

In [298]:
dados = pd.DataFrame()

for arquivo in arquivosord:
  tabela = pd.read_csv(f'{dir}/{arquivo}')
  tabela['Arquivo Origem'] = arquivo
  dados = pd.concat([dados,tabela])

dados.reset_index(drop=True,inplace=True)

In [299]:
dados.head()

,Data,Compra,Valor,Categoria,Arquivo Origem
0,01 DEZ,Uber* Trip,"R$ 12,99",Transporte,Nubank_2025-01-11.csv
1,04 DEZ,Amazon Marketplace - Parcela 2/7,"R$ 53,28",Shopping e ecommerce,Nubank_2025-01-11.csv
2,04 DEZ,Saoleopoldomeias - Parcela 2/4,"R$ 75,50",Shopping e ecommerce,Nubank_2025-01-11.csv
3,04 DEZ,Amazon Marketplace - Parcela 7/8,"R$ 53,13",Shopping e ecommerce,Nubank_2025-01-11.csv
4,04 DEZ,Amazon Marketplace - Parcela 3/6,"R$ 55,20",Shopping e ecommerce,Nubank_2025-01-11.csv


In [300]:
dados['Fech. Fatura'] = dados['Arquivo Origem']\
.str.rsplit('_', n=1).str[1]\
.str.split('.').str[0]

# Corrigindo e convertendo a coluna Valor para float 🔢


In [301]:
dados['Valor'] = dados['Valor']\
.str.replace('R$','')

In [302]:
dados['Valor'] = dados['Valor'].str.replace(',','.')
mask = pd.to_numeric(dados['Valor'], errors='coerce').isna()
dados[mask]

,Data,Compra,Valor,Categoria,Arquivo Origem,Fech. Fatura
880,13 AGO,IMOBILIARIA SAO LUIZ LTDA,1.011.55,Casa,Nubank_2025-09-11.csv,2025-09-11


In [303]:
# Explicação do REGEX: str.replace() substitui um ponto seguido (lookfoward)   (?=) de qualquer coisa .* mais um ponto \. Removendo, assim, somente o primeiro
dados['Valor'] = dados['Valor'].str.replace(r'\.(?=.*\.)', '', regex=True)

In [304]:
dados['Valor'] = dados['Valor'].astype(float)

# Corrigindo as datas das observações ⛴

In [305]:
dia_mes = dados['Data'].str.split(' ')
dados['Dia'] = dia_mes.str[0].str.strip()
dados['Mes'] = dia_mes.str[1].str.strip()
dados['Ano'] = dados['Fech. Fatura'].str\
              .split('-').str[0]\
              .str.strip()
meses = {
    "JAN": 1,
    "FEV": 2,
    "MAR": 3,
    "ABR": 4,
    "MAI": 5,
    "JUN": 6,
    "JUL": 7,
    "AGO": 8,
    "SET": 9,
    "OUT": 10,
    "NOV": 11,
    "DEZ": 12
}

dados['N Mes'] = dados['Mes'].map(meses)

dados['Data_Compra_Aux'] = dados.Ano + "-"\
                           + dados['N Mes'].astype(str) + "-"\
                           + dados.Dia

In [306]:
dados.drop(columns=['Dia','Mes','Ano','N Mes'],inplace=True)

In [307]:
dtdiff = pd.to_datetime(dados['Fech. Fatura'])-pd.to_datetime(dados['Data_Compra_Aux'])
dados['Dias_Compra'] = dtdiff.dt.days

In [308]:
dados.drop(columns=["Data"], inplace=True)

In [309]:
from dateutil.relativedelta import relativedelta

In [310]:
dados['Data'] = pd.to_datetime(dados['Data_Compra_Aux'])
filtro = dados['Dias_Compra'] < 0
dados.loc[filtro, 'Data'] = dados.loc[filtro, 'Data'] - pd.DateOffset(years=1)

In [311]:
dados.drop(columns=['Data_Compra_Aux', 'Dias_Compra'],inplace=True)

# Preparando amostra para o cálculo da entropia

1) Agrupar por mês comando as compras e criar uma coluna extra GastoMensal\
2) Criar coluna binária GastoAlto que será mascara de GastoMensal > 4600.00\
GastoAlto será o target\
3) Para cada coluna calcular a entropia \
4) Queremos saber qual característica tem maior IG (Information Gain), ou seja, qual varíavel tem maior significância na previsão se o GastoMensal ultrapassa R$400.00.

In [341]:
dados['Total Mes'] = (dados.groupby('Fech. Fatura')['Valor'].transform('sum'))
dados[-5:]

,Compra,Valor,Categoria,Arquivo Origem,Fech. Fatura,Data,Total Mes,GastoAlto
1065,•••• 2796 Uber* Trip,31.00,Transporte,Nubank_2025-11-11.csv,2025-11-11,2025-11-01,4221.48,False
1066,•••• 4725 Cicclo,17.90,Lavanderia,Nubank_2025-11-11.csv,2025-11-11,2025-11-01,4221.48,False
1067,•••• 2796 Uber* Trip,5.87,Transporte,Nubank_2025-11-11.csv,2025-11-11,2025-11-02,4221.48,False
1068,•••• 2796 Macromix Fl,126.86,Mercados,Nubank_2025-11-11.csv,2025-11-11,2025-11-02,4221.48,False
1069,•••• 2796 Ifd*Unidasul Distribui,131.72,Ifood,Nubank_2025-11-11.csv,2025-11-11,2025-11-03,4221.48,False


In [343]:
dados['GastoAlto'] = dados['Total Mes'] > 4600.00

In [342]:
gasto_abaixo = dados[dados['GastoAlto'] == False]

In [315]:
if gasto_abaixo.empty:
    print("Não meses com total abaixo do comparado.")
else:
    print(f"Há {len(gasto_abaixo['Fech. Fatura'].unique())} meses com total abaixo do comparado.")

Há 2 meses com total abaixo do comparado.


# Funções para entropia H() e ganho de informação IG

In [316]:
import numpy as np

def entropy_by_column(col):
    values, counts = np.unique(col, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

def information_gain(target, feature):

  target_entropy = entropy_by_column(target)
  feature_entropy = entropy_by_column(feature)

  information_gain = target_entropy - feature_entropy

  return information_gain

A entropia condicional, nesse caso, olha para uma coluna (ou uma feature) e calcula para os seus valores únicos (unique()) o efeito (ou o peso) sobre o grau de incerteza do target (GastoAlto). \
Ou seja, define o quanto a feature influencia no padrão do target.\
O target (GastoAlto) é uma coluna binária, que se define através do teste GastoMes > 4600 => True senão False.

In [317]:

def conditional_entropy(df, target, feature):
    feature_values = df[feature].unique()
    len_df = len(df)

    conditional_entropy = 0  # initialize accumulator

    for value in feature_values:
        df_feature_value = df[df[feature] == value]
        ft_weight = len(df_feature_value) / len_df
        target_entropy = entropy_by_column(df_feature_value[target])
        conditional_entropy += ft_weight * target_entropy

    return conditional_entropy

In [318]:
def information_gain(df, target, feature):
    target_entropy = entropy_by_column(df[target])
    conditional_ent = conditional_entropy(df, target, feature)
    return target_entropy - conditional_ent

A entropia do valor em característica analisará quanto ou qual a relevância do valor na previsão da coluna target. \
Ex. Quanto Categoria = 'Shopping e Ecommerce' pode nos dizer sobre o gasto mensal ter sido maior que 4600?\
É de conhecimento empírico que compras de shopping fazem o gasto mensal subir... mas vamos verificar na próxima secção.

In [319]:
def entropy_of_value(df, target, feature, value):
    subset = df[df[feature] == value]
    return entropy_by_column(subset[target]), len(subset)

# Calculando entropia de uma característica (feature ou coluna)

In [320]:
h_target = entropy_by_column(dados['GastoAlto'])
h_conditional = conditional_entropy(dados, 'GastoAlto', 'Categoria')
ig = information_gain(dados, 'GastoAlto', 'Categoria')

print(f"Entropia do target: {h_target:.4f}")
print(f"Entropia condicional: {h_conditional:.4f}")
print(f"Information Gain: {ig:.4f}")


Entropia do target: 0.6449
Entropia condicional: 0.6234
Information Gain: 0.0215


In [321]:
# Interpretação do ganho
if ig == 0:
    print("→ A feature NÃO reduz a incerteza sobre o target.")
elif ig < 0.05:
    print("→ Reduz a incerteza, mas muito pouco (quase irrelevante).")
elif ig < 0.15:
    print("→ Feature razoável para melhorar a previsão.")
elif ig < 0.30:
    print("→ Feature forte, reduz bem a incerteza.")
else:
    print("→ Feature MUITO forte, ajuda bastante a prever o target!")

→ Reduz a incerteza, mas muito pouco (quase irrelevante).


Uma situação diferente seria se calculassemos a entropia condicional de GastoMes e GastoAlto, como segue:\
h(GastoAlto|GastoMes) = 0\
Pois GastoMes impacta diretamente no target.\
E o ganho de informação seria IG = 1.\
Ou seja, GastoMes preve ou ajuda a prever GastoAlto com muita certeza.

# Calculando a entropia de um valor na categoria (feature ou coluna)

Retornamos ao exemplo:
Ex. Quanto Categoria = 'Shopping e Ecommerce' pode nos dizer sobre o gasto mensal ter sido maior que 4600?
É de conhecimento empírico que compras de shopping fazem o gasto mensal subir... mas vamos verificar na próxima secção.

In [322]:
h_shopping, n_shopping = entropy_of_value(dados, "GastoAlto", "Categoria", "Shopping e ecommerce")

print(f"H(GastoAlto | Categoria=Shopping e ecmmerce) = {h_shopping:.4f}")
print(f"N de compras em Shopping e Ecommerce: {n_shopping}")

H(GastoAlto | Categoria=Shopping e ecmmerce) = 0.6606
N de compras em Shopping e Ecommerce: 181


In [329]:
''' Como é só um valor para a característica o ganho é calculado assim: H(target) - H(Categoria|Valor).
H(Categoria|Valor) é calculado pela função entropy_by_value() acima.
'''

h_target = entropy_by_column(dados['GastoAlto'])

h_shopping = entropy_by_column(dados[dados["Categoria"] == "Shopping e ecommerce"]['GastoAlto'])

print(f"Entropia total: {h_target:.4f}")
print(f"Entropia no subset Shopping: {h_shopping:.4f}")

Entropia total: 0.6449
Entropia no subset Shopping: 0.6606


In [338]:
match h_shopping:
    case v if v >= 0.75:
        print("O valor NÃO prevê o alvo (entropia muito alta)")
    case v if v >= 0.50:
        print("O valor prevê pouco, quase como jogar uma moeda")
    case v if v >= 0.25:
        print("O valor prevê moderadamente o alvo")
    case v if v >= 0:
        print("O valor prevê bem o alvo (entropia baixa)")
    case _:
        print("Valor inválido")


O valor prevê pouco, quase como jogar uma moeda


Lembrando que:\
**Ganho de informação (Information Gain) nunca é definido para um único valor da feature, apenas para a feature inteira.**